# Feyisa automated water extraction index (AWEI)

From [Feyisa et al. (2014)](https://www.sciencedirect.com/science/article/pii/S0034425713002873)

Feyisa et al. have two indices, AWEIsh and AWEInsh, for images with and without shadows. Not sure how to incorporate both yet, because AWEI + BQA shadow bands don't eliminate shadows 100%. 

In [56]:
import ee; ee.Initialize()
import ipyleaflet  # an interactive mapping "widget" for the notebook

### Detect floods

In [81]:
scene = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044033_20170105')

# AWEI Function from Feyisa et al. (2014) to detect water
def feyisa(image):
    return image.expression("4 * (b('B3')-b('B6')) - (0.25 * b('B5') + 2.75 * b('B7'))").gt(0)

### Remove clouds, cloud shadows, and snow

In [82]:
# Compute the bits we need to extract
def getQABits(image, start, end, newName):
    pattern = 0
    i = start
    for i in range(start, end+1):
        pattern += 2**i
    # Return a single band image of the extracted QA bits, giving the band a new name
    return image.select([0], [newName])\
              .bitwiseAnd(pattern)\
              .rightShift(start);

# Cloud shadows QA (Landsat 8, T1 TOA)
def cloud_shadowsQA(image):
    QA = image.select(['BQA'])
    return getQABits(QA, 7, 8, 'cloud_shadows').eq(1)

# Clouds QA (Landsat 8, T1 TOA)
def cloudsQA(image):
    QA = image.select(['BQA'])
    return getQABits(QA, 4, 4, 'clouds').eq(0)

def snowQA(image):
    QA = image.select(['BQA'])
    return getQABits(QA, 9, 10, 'snow_ice').eq(1)

In [89]:
cs = cloud_shadowsQA(scene)
c = cloudsQA(scene)
s = snowQA(scene)

sceneFlood = feyisa(scene).rename('floodBinary') # Detect water
sceneFlood = sceneFlood.updateMask(cs).updateMask(c).updateMask(s) # Mask out clouds, cloud shadows, and snow/ice
sceneFlood = sceneFlood.updateMask(sceneFlood.eq(1)) # Mask out non-water for visualizing

### Map the floods 

In [90]:
def GetTileLayerUrl(ee_image_object):
    map_id = ee.Image(ee_image_object).getMapId()
    tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
    return tile_url_template.format(**map_id)

# Get centroid coords to center the map
center = scene.geometry().centroid().coordinates().getInfo()

# Visualization parameters
vizFlood = {'palette':['red'],
           'opacity':0.5}

map1 = ipyleaflet.Map(
    center=(center[1],center[0]), zoom=8,
    layout={'height':'400px'}
)
map1.add_layer(
    ipyleaflet.TileLayer(url=GetTileLayerUrl(
        sceneFlood.visualize(**vizFlood)
    )
))
map1

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …